In [32]:
import tensorflow as tf 
import matplotlib.pyplot as plt 
import numpy as np 
from glob import glob 
import random 
import os
from PIL import Image


In [33]:
# paths = '20220318/dataset'
paths = 'dataset'
# image_path = os.path.join(paths)
images = glob(paths + '/*/*')
# imagesbb

In [38]:
images

['dataset\\cane\\OIF-e2bexWrojgtQnAPPcUfOWQ.jpeg',
 'dataset\\cane\\OIP---A27bIBcUgX1qkbpZOPswHaFS.jpeg',
 'dataset\\cane\\OIP---cByAiEbIxIAleGo9AqOQAAAA.jpeg',
 'dataset\\cane\\OIP---ZIdwfUcJeVxnh47zppcQHaFj.jpeg',
 'dataset\\cane\\OIP---ZRsOF7zsMqhW30WeF8-AHaFj.jpeg',
 'dataset\\cane\\OIP---_cJbI6Ei26w5bW1urHewHaCf.jpeg',
 'dataset\\cane\\OIP--0C2kq6I68beB3DduLoxMgHaEf.jpeg',
 'dataset\\cane\\OIP--0CZnfLuBERVtzRm2QbB3gHaK4.jpeg',
 'dataset\\cane\\OIP--1QXriWyOTJg-9fEwbznmgHaI4.jpeg',
 'dataset\\cane\\OIP--2uTtzjaszBfEMCROuLN3wHaLH.jpeg',
 'dataset\\cane\\OIP--2z_zAuTMzgYM_KynUl9CQHaE7.jpeg',
 'dataset\\cane\\OIP--565fkAJy3DoFs81C9uE1QHaEY.jpeg',
 'dataset\\cane\\OIP--6mVvk2SFEi7TxqtyMbdvgHaEK.jpeg',
 'dataset\\cane\\OIP--6UY_QQC69uZF4CrwEcfUQHaHa.jpeg',
 'dataset\\cane\\OIP--7xBAVJkrjDLNCknR2fuKgHaHa.jpeg',
 'dataset\\cane\\OIP--8oCad9Nr69Ol4YEkvXJFQHaNK.jpeg',
 'dataset\\cane\\OIP--90_fw5Y28UC44KVSPuxdAHaJ3.jpeg',
 'dataset\\cane\\OIP--9pxEn5HUPsqA38ao3O7TgHaGV.jpeg',
 'dataset\\can

In [35]:
RES = 100 
N_CLASS = 10
BATCH_SIZE = 64

# 26179 images 
N_TRAIN = 20000
N_VAL = 6179 

class_list = ['cavallo',
 'pecora',
 'elefante',
 'gatto',
 'scoiattolo',
 'gallina',
 'ragno',
 'mucca',
 'cane',
 'farfalla']

In [5]:
def to_categorical(classes):
    idx = class_list.index(classes)
    return np.array(tf.keras.utils.to_categorical(idx,10), dtype=np.uint8)

In [36]:
train_images = [] 
train_labels = [] 

test_images = [] 
test_labels = [] 

In [41]:
ttest = np.array(Image.open('dataset\\cane\\OIF-e2bexWrojgtQnAPPcUfOWQ.jpeg').convert('RGB').resize((RES, RES)))

ttest.min(), ttest.max()

(0, 254)

In [42]:
for image in images[:N_TRAIN]: 
#     print(image)
    img = Image.open(image).convert('RGB').resize((RES, RES))
    img = np.array(img) / 254. 
    
    lbl = image.split('\\')[1]
    lbl = to_categorical(lbl)
    # lbl = class_list.index(lbl)
    
    train_images.append(img) 
    train_labels.append(lbl) 
    

In [43]:
for image in images[N_TRAIN:]: 
    img = Image.open(image).convert('RGB').resize((RES, RES))
    img = np.array(img) / 254. 
    
    lbl = image.split('\\')[1]
    lbl = to_categorical(lbl)
    # lbl = class_list.index(lbl)
    
    test_images.append(img) 
    test_labels.append(lbl) 
    

In [44]:
train_images = np.array(np.reshape(train_images, (len(train_images), RES, RES, 3)))
train_labels = np.array(np.reshape(train_labels, (len(train_labels), 10)))

test_images = np.array(np.reshape(test_images, (len(test_images), RES, RES, 3)))
test_labels = np.array(np.reshape(test_labels, (len(test_labels), 10)))
    
    
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

C:\Users\Sehwan Yoo\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


ValueError: cannot reshape array of size 35795 into shape (35795,10)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [ ]:
train_ds = train_ds.shuffle(100).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [ ]:
# def preprocessing(image, label):
#     # image = tf.cast(image / 255., tf.uint8) 
#     image = tf.cast(image / 255., tf.uint8) 
# #     print(image.dtype, label.dtype)
#     return image, label

def aug(image, label):
    image = tf.image.random_crop(image, [RES, RES, 3])
    image = tf.image.random_flip_left_right(image)
    return image, label

In [ ]:
train_ds = train_ds.map(aug).shuffle(100).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(aug).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [ ]:
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
mobilenetv3 = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(RES, RES, 3))

In [ ]:
def create_model():
    inputs = tf.keras.Input(shape=(RES, RES, 3))
    x = preprocess_input(inputs) 
    x = mobilenetv3(x) 
    x = tf.keras.layers.GlobalAveragePooling2D()(x) 
    x = tf.keras.layers.Dense(N_CLASS)(x)
    x = tf.keras.layers.BatchNormalization()(x) 
    x = tf.keras.layers.Softmax()(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)
    
    return model

In [ ]:
model = create_model()

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
MobilenetV3small (Functional (None, 1, 1, 1024)        1529968   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                10250     
_________________________________________________________________
batch_normalization (BatchNo (None, 10)                40        
_________________________________________________________________
softmax (Softmax)            (None, 10)                0         
Total params: 1,540,258
Trainable params: 1,528,126
Non-trainable params: 12,132
______________________________________________

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=['accuracy'])

In [ ]:
for img, lbl in train_ds.take(1):
    print(img.shape, img.dtype)
    # print(max(img), min(img))
    print(lbl)

(64, 100, 100, 3) <dtype: 'float64'>
tf.Tensor(
[[0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0

In [ ]:
hist = model.fit(train_ds, epochs=30, validation_data=(test_ds))

Epoch 1/30
312/312 [==============================] - 160s 494ms/step - loss: 2.3168 - accuracy: 0.1796 - val_loss: 2.4275 - val_accuracy: 0.0000e+00
Epoch 2/30
312/312 [==============================] - 156s 498ms/step - loss: 2.2465 - accuracy: 0.2478 - val_loss: 2.6326 - val_accuracy: 0.0000e+00
Epoch 3/30
312/312 [==============================] - 151s 484ms/step - loss: 2.2119 - accuracy: 0.2435 - val_loss: 2.6785 - val_accuracy: 0.0000e+00
Epoch 4/30
312/312 [==============================] - 158s 506ms/step - loss: 2.1856 - accuracy: 0.2436 - val_loss: 2.8409 - val_accuracy: 0.0000e+00
Epoch 5/30
312/312 [==============================] - 151s 484ms/step - loss: 2.1655 - accuracy: 0.2436 - val_loss: 2.9146 - val_accuracy: 0.0000e+00
Epoch 6/30
168/312 [===============>..............] - ETA: 1:04 - loss: 2.0247 - accuracy: 0.4523

KeyboardInterrupt: 

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()